# Mission 1: Create a AI Model
Description
Download the “Monatszahlen Verkehrsunfälle” Dataset from the München Open Data Portal. Here you see the number of accidents for specific categories per month. Important are the first 5 columns:
* Category
* Accident-type (insgesamt means total for all subcategories)
* Year
* Month
* Value

Your goal would be to visualise historically the number of accidents per category (column1). The dataset currently contains values until the end of 2020. Create an application that forecasts the values for:

* Category: 'Alkoholunfälle'
* Type: 'insgesamt
* Year: '2021'
* Month: '01'


Note: There has been recently some update on the data shared with you for the AI Engineering Challenge. The dataset has recently been updated and includes values until 2021. You should drop the records which come after 2020 for developing the prediction model. If you are interested, you can compute the error between your prediction values and the actual numbers (ground truth data).

## 1. Loading the Data

In [1]:
import pandas as pd

### LOADING THE DATA IN DATAFRAME
df = pd.read_csv('monatszahlen2412_verkehrsunfaelle_06_12_24.csv')
df.head()

,MONATSZAHL,AUSPRAEGUNG,JAHR,MONAT,WERT,VORJAHRESWERT,VERAEND_VORMONAT_PROZENT,VERAEND_VORJAHRESMONAT_PROZENT,ZWOELF_MONATE_MITTELWERT
0,Alkoholunfälle,insgesamt,2024,202401,NaN,NaN,NaN,NaN,NaN
1,Alkoholunfälle,insgesamt,2024,202402,NaN,NaN,NaN,NaN,NaN
2,Alkoholunfälle,insgesamt,2024,202403,NaN,NaN,NaN,NaN,NaN
3,Alkoholunfälle,insgesamt,2024,202404,NaN,NaN,NaN,NaN,NaN
4,Alkoholunfälle,insgesamt,2024,202405,NaN,NaN,NaN,NaN,NaN


In [3]:
df.columns

Index(['MONATSZAHL', 'AUSPRAEGUNG', 'JAHR', 'MONAT', 'WERT', 'VORJAHRESWERT',
       'VERAEND_VORMONAT_PROZENT', 'VERAEND_VORJAHRESMONAT_PROZENT',
       'ZWOELF_MONATE_MITTELWERT'],
      dtype='object')

Filtering the dataset for the desired category

```
Category: 'Alkoholunfälle'
Type: 'insgesamt
```

Because we have to make predictions for this category in the end

In [ ]:
# Filter for the desired category and type
mask = (df['MONATSZAHL'] == 'Alkoholunfälle') & (df['AUSPRAEGUNG'] == 'insgesamt')
df_filtered = df[mask].copy()

In [5]:
df_filtered.head()

,MONATSZAHL,AUSPRAEGUNG,JAHR,MONAT,WERT,VORJAHRESWERT,VERAEND_VORMONAT_PROZENT,VERAEND_VORJAHRESMONAT_PROZENT,ZWOELF_MONATE_MITTELWERT
0,Alkoholunfälle,insgesamt,2024,202401,NaN,NaN,NaN,NaN,NaN
1,Alkoholunfälle,insgesamt,2024,202402,NaN,NaN,NaN,NaN,NaN
2,Alkoholunfälle,insgesamt,2024,202403,NaN,NaN,NaN,NaN,NaN
3,Alkoholunfälle,insgesamt,2024,202404,NaN,NaN,NaN,NaN,NaN
4,Alkoholunfälle,insgesamt,2024,202405,NaN,NaN,NaN,NaN,NaN


## 2. Preprocessing The Data

We only want rows where the 'MONAT' is in the form 'YYYYMM' (not 'Summe')

Why?

The 'MONAT' column contains values like '202201', '202202', etc., representing specific months in the format 'YYYYMM' (e.g., January 2022 is '202201').   

However, some rows have 'MONAT' values like 'Summe', which represent aggregate data (e.g., total accidents for the year 2022).

Time series forecasting models, such as ARIMA or SARIMA, require consistent and uniformly spaced data points. Including aggregate rows like 'Summe' disrupts this consistency because they don't represent the same temporal granularity as the monthly data

Aggregate or summary rows can contaminate the dataset, leading to misleading patterns. For example, the 'Summe' row aggregates all monthly data for the year, which can introduce spikes or drops that don't align with the actual monthly trends.

In [6]:
df_filtered = df_filtered[df_filtered['MONAT'].apply(lambda x: str(x).isdigit())]
df_filtered['MONAT'] = df_filtered['MONAT'].astype(str)

# Extract year and month
df_filtered['year'] = df_filtered['JAHR'].astype(int)
df_filtered['month'] = df_filtered['MONAT'].str[-2:].astype(int)

df_filtered['date'] = pd.to_datetime(df_filtered['year'].astype(str) + '-' + df_filtered['month'].astype(str) + '-01')

df_filtered = df_filtered.sort_values('date')

# Ensure 'WERT' is numeric and handle missing values
df_filtered['WERT'] = pd.to_numeric(df_filtered['WERT'], errors='coerce')
df_filtered = df_filtered.dropna(subset=['WERT'])

df_filtered.set_index('date', inplace=True)

df_filtered.head()

,MONATSZAHL,AUSPRAEGUNG,JAHR,MONAT,WERT,VORJAHRESWERT,VERAEND_VORMONAT_PROZENT,VERAEND_VORJAHRESMONAT_PROZENT,ZWOELF_MONATE_MITTELWERT,year,month
date,,,,,,,,,,,
2000-01-01,Alkoholunfälle,insgesamt,2000,200001,78.0,NaN,NaN,NaN,78.0,2000,1
2000-02-01,Alkoholunfälle,insgesamt,2000,200002,53.0,NaN,-32.05,NaN,66.0,2000,2
2000-03-01,Alkoholunfälle,insgesamt,2000,200003,73.0,NaN,37.74,NaN,68.0,2000,3
2000-04-01,Alkoholunfälle,insgesamt,2000,200004,78.0,NaN,6.85,NaN,71.0,2000,4
2000-05-01,Alkoholunfälle,insgesamt,2000,200005,96.0,NaN,23.08,NaN,76.0,2000,5


In [ ]:
# Detect and Handle Outliers
# Using the IQR method to detect outliers
Q1 = df_filtered['WERT'].quantile(0.25)
Q3 = df_filtered['WERT'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = df_filtered[(df_filtered['WERT'] < lower_bound) | (df_filtered['WERT'] > upper_bound)]
print("\nDetected Outliers:")
outliers


Detected Outliers:


,MONATSZAHL,AUSPRAEGUNG,JAHR,MONAT,WERT,VORJAHRESWERT,VERAEND_VORMONAT_PROZENT,VERAEND_VORJAHRESMONAT_PROZENT,ZWOELF_MONATE_MITTELWERT,year,month
date,,,,,,,,,,,
2000-05-01,Alkoholunfälle,insgesamt,2000,200005,96.0,NaN,23.08,NaN,76.0,2000,5
2000-07-01,Alkoholunfälle,insgesamt,2000,200007,99.0,NaN,73.68,NaN,76.0,2000,7
2001-09-01,Alkoholunfälle,insgesamt,2001,200109,107.0,84.0,78.33,27.38,73.0,2001,9
2002-09-01,Alkoholunfälle,insgesamt,2002,200209,107.0,107.0,50.70,0.00,69.0,2002,9


In [12]:
import numpy as np

# Replace outliers with capped values (capping)
df_filtered['WERT'] = np.where(
    df_filtered['WERT'] < lower_bound,
    lower_bound,
    np.where(
        df_filtered['WERT'] > upper_bound,
        upper_bound,
        df_filtered['WERT']
    )
)

In [13]:
df_filtered.loc[outliers.index]

,MONATSZAHL,AUSPRAEGUNG,JAHR,MONAT,WERT,VORJAHRESWERT,VERAEND_VORMONAT_PROZENT,VERAEND_VORJAHRESMONAT_PROZENT,ZWOELF_MONATE_MITTELWERT,year,month
date,,,,,,,,,,,
2000-05-01,Alkoholunfälle,insgesamt,2000,200005,87.5,NaN,23.08,NaN,76.0,2000,5
2000-07-01,Alkoholunfälle,insgesamt,2000,200007,87.5,NaN,73.68,NaN,76.0,2000,7
2001-09-01,Alkoholunfälle,insgesamt,2001,200109,87.5,84.0,78.33,27.38,73.0,2001,9
2002-09-01,Alkoholunfälle,insgesamt,2002,200209,87.5,107.0,50.70,0.00,69.0,2002,9
